In [5]:
!pip install matplotlib seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [6]:
import os
import pickle
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
def load_data(input_file):
    df = pd.read_csv(input_file)
    X = df['cleaned_text'].fillna('')
    y = df['risk_level']
    return X, y

In [11]:
def evaluate_models(data_path, models_dir):
    print(f"Loading data from {data_path}...")
    X, y = load_data(data_path)

    # Define model path FIRST
    model_path = os.path.join(models_dir, 'best_model.pkl')

    if not os.path.exists(model_path):
        print("Best model not found. Run training first.")
        return

    # Load Pipeline model
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    print(f"Evaluating Best Model: {type(model).__name__}...")

    from sklearn.model_selection import train_test_split

    # Split RAW TEXT (Pipeline handles TF-IDF internally)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )

    # Predict
    y_pred = model.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f"\nTest Accuracy: {acc:.4f}")

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=sorted(y.unique()),
        yticklabels=sorted(y.unique())
    )

    plt.xlabel('Predicted')
    plt.ylabel('Ground Truth')
    plt.title('Confusion Matrix')

    save_path = os.path.join(models_dir, 'confusion_matrix.png')
    plt.savefig(save_path)
    plt.close()

    print(f"\nConfusion Matrix saved to {save_path}")

if __name__ == "__main__":
    try:
        base_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    except NameError:
        base_dir = os.path.dirname(os.getcwd()) #Gets current file location
    input_path = os.path.join(base_dir, "data", "processed", "processed_contracts.csv")
    models_dir = os.path.join(base_dir, "models")
    
    evaluate_models(input_path, models_dir)


Loading data from /Users/ash/CascadeProjects/projects..../contrack_risk_analyser/data/processed/processed_contracts.csv...
Evaluating Best Model: Pipeline...

Test Accuracy: 0.9318

Classification Report:
              precision    recall  f1-score   support

        High       0.87      0.91      0.89       542
         Low       0.97      0.96      0.96      1126
      Medium       0.92      0.91      0.92       868

    accuracy                           0.93      2536
   macro avg       0.92      0.93      0.92      2536
weighted avg       0.93      0.93      0.93      2536


Confusion Matrix saved to /Users/ash/CascadeProjects/projects..../contrack_risk_analyser/models/confusion_matrix.png
